The first example notebook introduces some of the library's basic functionalities for a simple use case. This use case is the non-parametric inference of the drift function of a stochastic process, given its diffusion function. The necessary data is provided in the form of perturbed mean exit time values from an Ornstein-Uhlenbeck process. As of now, the computations are restricted to one-dimensional spacial domains.

The presented steps include data generation, linearized inference and the visualization of the results.

In [1]:
import os
import numpy as np

from sp_inference import processes, model, logging

The configuration is centered around python dictionaries and numpy data structures. Most dictionaries serve as direct input for the library components, whereas the remaining dicts are just for concise representation of the settings.

In [2]:
# Settings of the logger, which is shared by all library components
# The data is saved into a pre-defined file structure to reduce the number of settings
logSettings = {
    # Output folder; if None is specified, no data is saved
    "output_directory":           "01_example_single_parameter",
    # Decides if log info is printed to the screen
    "verbose":                    True
}

# Settings for the generation of data from a prototypical stochastic process
dataSettings = {
    # Process type, check the 'processes' module for options
    "process_type":               "OUProcess",
    # Parameter(s) for the drift function (linear for Ornstein-Uhlenbeck)
    "drift_parameters":           1,
    # Parameter(s) for the diffusion function (constant for Ornstein-Uhlenbeck)
    "diffusion_parameters":       1,
    # Standard deviation of the zero-centered Gaussian noise on the generated data
    "standard_deviation":         0.02,
    # RNG seed for data point generation
    "rng_seed":                   0,
    # Number of data points to generate
    "num_points":                 50,
    # Spacial domain on which to compute data (here for the mean exit time)
    "domain_bounds":              [-1, 1]
}

# Settings for the inference model
modelSettings = {
    # Which parameter(s) to infer, can be 'drift', 'diffusion', or 'all'
    "params_to_infer":            "drift",
    # Model type describing the PDE constraint of the inference problem and source of the data.
    # For options have a look at the implemented 'forms' in the 'pde_problems' module
    "model_type":                 "mean_exit_time",
    # Determines if the underlying pde problem is treated as stationary or transient
    # All models can possibly be made transient, but for e.g. the mean exit time problem this would
    # lead to errornous results.
    "is_stationary":              True
}

# Settings of the Bi-Laplacian prior transferred from the hIPPYlib library
priorSettings = {
    # Mean function (for the drift)
    "mean_function":              lambda x: np.ones(x.shape),
    # Parameter for the covariance operator
    "gamma":                      1,
    # Parameter for the covariance operator
    "delta":                      2,
    # Determines if Robin boundary conditions are used for the computation of the covariance field
    "robin_bc":                   False
}

# Settings for the FEM solver of the PDE constraint used for the inference problem
feSettings = {
    # Number of mesh points
    "num_mesh_points":            500,
    # Locations of the spacial domain boundaries
    "boundary_locations":         [-1, 1],
    # Dirichlet boundary values (homogeneous for mean exit time problem)
    "boundary_values":            [0, 0],
    # FEM element degrees for PDE solution (+ adjoint) and parameter function
    "element_degrees":            [1, 1],
    # Pre-determined diffusion function that is not inferred from the data
    "squared_diffusion_function": lambda x: np.ones(x.shape)
}

# Settings of the solver for the linearized inference problem (MAP).
# This is an inexact Newton-CG solver with Armijo line search for globalization
solverSettings = {
    # Relative termination tolerance in the objective functional gradient norm
    "rel_tolerance":              1e-6,
    # Relative termination tolerance in the objective functional gradient norm
    "abs_tolerance":              1e-12,
    # Maximum number of iterations
    "max_iter":                   50,
    # Number of Gauss Newton iterations before switching to Newton
    "GN_iter":                    5,
    # Armijo constant for sufficient reduction
    "c_armijo":                   1e-4,
    # Maximum number of backtracking iterations during line search
    "max_backtracking_iter":      10
}

# Settings for the construction of the reduced Hessian of the linearized problem about the MAP
hessianSettings = {
    # Number of generalized eigenvalue/-vector pairs to include
    "num_eigvals":                20,
    # Number of value to oversample for robustness of the randomized algorithm
    "num_oversampling":           5
}

In [3]:
try:
    os.system('rm -r ' + logSettings["output_directory"])
except:
    pass

# The logger is instantiated as a separate object that is passed to other components
logger = logging.Logger(logSettings["verbose"],
                        logSettings["output_directory"])

We firstly create artificial data from a prototypical stochastic process. The spacial locations of the data points are passed to the generating routine for higher flexibility. The data is generated by superimposing zero-centered Gaussian noise to the exact value of the generating model, here the solution of the process's mean exit time problem. The resulting data is used for the definition of the misfit functional of the inference problem.


In [4]:
randGenerator = np.random.default_rng(dataSettings["rng_seed"])
randLocs = randGenerator.uniform(*dataSettings["domain_bounds"], dataSettings["num_points"])
dataSettings["domain_points"] = randLocs

processType = processes.get_process(dataSettings["process_type"])
process = processType(dataSettings["drift_parameters"],
                      dataSettings["diffusion_parameters"],
                      logger)

# Generating function returns exact and perturbed data
forwardNoisy, forwardExact = process.generate_data(modelSettings["model_type"],
                                                   modelSettings["is_stationary"],
                                                   dataSettings)

exactDrift = process.compute_drift(randLocs)
exactParamData = [randLocs, exactDrift]
randForwardData = [randLocs, forwardNoisy]
exactForwardData = [randLocs,  forwardExact]

# New settings dict for the misfit functional
misFitSettings = {
    "data_locations": randLocs,
    "data_values": forwardNoisy,
    "data_var": dataSettings["standard_deviation"]**2
}

===================== Invoke OUProcess =====================
 
Drift Coefficient(s): 1 
Diffusion Coefficient(s): 1 

Generate MET data:                 Successful 



The inference model simply takes settings for the overall model, its prior, the FEM setup and the misfit.

In [5]:
inferenceModel = model.SDEInferenceModel(modelSettings,
                                         priorSettings,
                                         feSettings,
                                         misFitSettings,
                                         logger=logger)

# Get drift mean and variance, along with mean exit time solution when using prior info
priorMeanData, priorVarianceData, priorForwardData = inferenceModel.get_prior_info("Randomized")

================== Invoke Inference Model ==================
 
Construct PDE Problem:             Successful 

Construct Prior:                   Successful 

Construct Misfit:                  Successful 

 
Calling FFC just-in-time (JIT) compiler, this may take some time.


An optimization routine computes the maximum a posteriori (MAP) estimate for the linearized problem. The algorithm employs a second order inexact Newton-CG algorithm to find the optimum. As a result, it returns the MAP point along with the reduced Hessian at that point (only considering a limited number of dominating eigenvalue-eigenvector pairs).

In [6]:
# Data for mean, variance and forward solution is returned in form of x-y-value pairs over the spacial domain
mapMeanData, mapVarianceData, mapForwardData, hessEigVals \
    = inferenceModel.compute_gr_posterior(solverSettings, hessianSettings)

=============== Conduct Linearized Inference ===============
 
Solve for MAP: 
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.

It  cg_it cost            misfit          reg             (g,dm)          ||g||L2        alpha          tolcg         
  1   1    6.494313e+03    6.477141e+03    1.717165e+01   -3.894909e+04   2.006017e+04   1.000000e+00   5.000000e-01
  2   1    2.386669e+02    2.261023e+02    1.256467e+01   -1.166931e+04   1.778806e+04   1.000000e+00   5.000000e-01
  3   2    6.879075e+01    5.955069e+01    9.240058e+00   -4.090203e+02   2.180025e+03   1.000000e+00   3